# 📦 FTL Threshold Cost Simulation for 1VNL (Vinyl)
Simulate how changing the FTL threshold affects average freight cost per pound.

In [44]:
import pandas as pd
# Load your dataset here
# Example: 
df = pd.read_csv("../../data/downloads/1vnl.csv")
# Make sure 'freight_price' and 'total_quantity' columns are present
df2 = pd.read_csv("../../data/downloads/1cbl.csv")

In [16]:
df.columns

Index(['site', 'invoice_id', 'freight_price', 'unique_commodity_group_input',
       'unique_commodity_description_input', 'total_estimated_area_cost',
       'total_estimated_cwt_cost', 'total_est_lbs', 'total_est_sqyd',
       'unique_commodity_group_output', 'unique_commodity_description_output',
       'total_cost', 'total_quantity', 'UOM', 'freight_ratio', 'market_rate',
       'xgs_rate', 'rate_ratio', 'shipment_type', 'outlier_flag', 'savings',
       'action'],
      dtype='object')

In [20]:
df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

In [26]:
df.replace([float('inf'), -float('inf')], 0, inplace=True)

In [18]:
df[['shipment_type','market_rate']]

,shipment_type,market_rate
0,LTL,0.05
1,LTL,0.24
2,LTL,0.05
3,LTL,0.64
4,LTL,0.9
...,...,...
1788,LTL,0.02
1789,LTL,0.28
1790,LTL,0.06
1791,LTL,0.02


In [ ]:
# 1VNL FTL and LTL analysis
def simulate_ftl_thresholds_verbose(data, freight_price_col='freight_price', quantity_col='total_quantity', thresholds=[10000, 15000, 20000, 25000]):
    data['quantity'] = pd.to_numeric(data[quantity_col], errors='coerce')
    data['freight_price'] = pd.to_numeric(data[freight_price_col], errors='coerce')

    results = []
    
    for threshold in thresholds:
        data['simulated_type'] = data['quantity'].apply(lambda q: 'FTL' if q >= threshold else 'LTL')
        data['unit_cost'] = (data['freight_price'] / data['quantity']).round(2)
        
        print('LTL:', data[data['simulated_type'] == 'LTL']['unit_cost'].mean())
        print('FTL:', data[data['simulated_type'] == 'FTL']['unit_cost'].mean())
        
        grouped = data.groupby('simulated_type').agg(
            avg_cost_per_lb=('unit_cost', 'mean'),
            shipment_count=('unit_cost', 'count')
        ).reset_index()
        grouped['threshold'] = threshold
        results.append(grouped)
    
    combined = pd.concat(results)
    pivot = combined.pivot(index='threshold', columns='simulated_type', values=['avg_cost_per_lb', 'shipment_count']).round(3)
    pivot.columns = [f"{metric}_{stype}" for metric, stype in pivot.columns]
    return pivot.reset_index()


In [43]:
valid_rows = df[(df['freight_price'] > 0) & (df['quantity'] > 0)]
summary_df = simulate_ftl_thresholds_verbose(valid_rows, 'freight_price', 'total_quantity')
display(summary_df)

LTL: 1.2619809637120718
FTL: 0.07495238095238091
LTL: 1.239107351225199
FTL: 0.07541666666666663
LTL: 1.2297800925925872
FTL: 0.07241379310344823
LTL: 1.2223519263944744
FTL: 0.07638297872340424


C:\Users\nzhuw\AppData\Local\Temp/ipykernel_9424/177666260.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['quantity'] = pd.to_numeric(data[quantity_col], errors='coerce')
C:\Users\nzhuw\AppData\Local\Temp/ipykernel_9424/177666260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['freight_price'] = pd.to_numeric(data[freight_price_col], errors='coerce')
C:\Users\nzhuw\AppData\Local\Temp/ipykernel_9424/177666260.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,threshold,avg_cost_per_lb_FTL,avg_cost_per_lb_LTL,shipment_count_FTL,shipment_count_LTL
0,10000,0.075,1.262,105.0,1681.0
1,15000,0.075,1.239,72.0,1714.0
2,20000,0.072,1.230,58.0,1728.0
3,25000,0.076,1.222,47.0,1739.0


In [47]:
df2.columns

Index(['site', 'invoice_id', 'freight_price', 'unique_commodity_group_input',
       'unique_commodity_description_input', 'total_estimated_area_cost',
       'total_estimated_cwt_cost', 'total_est_lbs', 'total_est_sqyd',
       'unique_commodity_group_output', 'unique_commodity_description_output',
       'total_cost', 'total_quantity', 'UOM', 'freight_ratio', 'market_rate',
       'xgs_rate', 'rate_ratio', 'shipment_type', 'outlier_flag', 'savings',
       'action'],
      dtype='object')

In [51]:
# 1CBL FTL and LTL analysis
def simulate_ftl_thresholds_cbl(data, freight_price_col='freight_price', quantity_col='total_quantity', thresholds_in_rolls=[20,25,30,45, 50, 60]):
    # Convert thresholds from rolls to SQYD
    thresholds = [rolls * 100 for rolls in thresholds_in_rolls]
    
    # Ensure numeric data
    data['quantity'] = pd.to_numeric(data[quantity_col], errors='coerce')
    data['freight_price'] = pd.to_numeric(data[freight_price_col], errors='coerce')
    
    results = []

    for threshold, rolls in zip(thresholds, thresholds_in_rolls):
        data['simulated_type'] = data['quantity'].apply(lambda q: 'FTL' if q >= threshold else 'LTL')
        data['unit_cost'] = (data['freight_price'] / data['quantity']).round(2)

        print(f"\nThreshold: {rolls} rolls ({threshold} SQYD)")
        print("LTL Avg Unit Cost:", data[data['simulated_type'] == 'LTL']['unit_cost'].mean())
        print("FTL Avg Unit Cost:", data[data['simulated_type'] == 'FTL']['unit_cost'].mean())
        
        grouped = data.groupby('simulated_type').agg(
            avg_cost_per_sqyd=('unit_cost', 'mean'),
            shipment_count=('unit_cost', 'count')
        ).reset_index()
        grouped['threshold_rolls'] = rolls
        results.append(grouped)
    
    combined = pd.concat(results)
    pivot = combined.pivot(index='threshold_rolls', columns='simulated_type', values=['avg_cost_per_sqyd', 'shipment_count']).round(3)
    pivot.columns = [f"{metric}_{stype}" for metric, stype in pivot.columns]
    return pivot.reset_index()


In [52]:
valid_rows = df2[(df2['freight_price'] > 0) & (df2['total_quantity'] > 0)]
summary_df = simulate_ftl_thresholds_cbl(valid_rows, 'freight_price', 'total_quantity')
display(summary_df)


Threshold: 20 rolls (2000 SQYD)
LTL Avg Unit Cost: 1.4342372881355934
FTL Avg Unit Cost: 0.41400000000000003

Threshold: 25 rolls (2500 SQYD)
LTL Avg Unit Cost: 1.4297058823529412
FTL Avg Unit Cost: 0.29374999999999996

Threshold: 30 rolls (3000 SQYD)
LTL Avg Unit Cost: 1.427907949790795
FTL Avg Unit Cost: 0.19285714285714284

Threshold: 45 rolls (4500 SQYD)
LTL Avg Unit Cost: 1.4239583333333334
FTL Avg Unit Cost: 0.145

Threshold: 50 rolls (5000 SQYD)
LTL Avg Unit Cost: 1.4239583333333334
FTL Avg Unit Cost: 0.145

Threshold: 60 rolls (6000 SQYD)
LTL Avg Unit Cost: 1.4239583333333334
FTL Avg Unit Cost: 0.145


C:\Users\nzhuw\AppData\Local\Temp/ipykernel_9424/510822632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['quantity'] = pd.to_numeric(data[quantity_col], errors='coerce')
C:\Users\nzhuw\AppData\Local\Temp/ipykernel_9424/510822632.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['freight_price'] = pd.to_numeric(data[freight_price_col], errors='coerce')
C:\Users\nzhuw\AppData\Local\Temp/ipykernel_9424/510822632.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,threshold_rolls,avg_cost_per_sqyd_FTL,avg_cost_per_sqyd_LTL,shipment_count_FTL,shipment_count_LTL
0,20,0.414,1.434,10.0,236.0
1,25,0.294,1.430,8.0,238.0
2,30,0.193,1.428,7.0,239.0
3,45,0.145,1.424,6.0,240.0
4,50,0.145,1.424,6.0,240.0
5,60,0.145,1.424,6.0,240.0
